In [9]:
import pymysql
import pandas as pd

# MySQL 연결
conn = pymysql.connect(
    host="database-4.cz02m00ykp2i.ap-northeast-2.rds.amazonaws.com",
    port=3306,
    user="admin",
    password="k33470115!",
    database="my_database"
)

# 데이터 가져오기
query = "SELECT * FROM train;"
df = pd.read_sql(query, conn)

# 연결 종료
conn.close()

# 데이터 확인
# print(df.info())
# print(df.head())

C:\Users\human\AppData\Local\Temp\ipykernel_15524\2777383118.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [10]:
df

,order_id,delivery_date,departure_gu,departure_dong,departure_lat,departure_lng,arrival_gu,arrival_dong,arrival_lat,arrival_lng,...,driver_longitude,refrigeration_available,fragile_item_available,hazardous_material_available,driver_max_delivery_distance_km,driver_max_delivery_weight,weekly_day_off_encoded,communication_encoded,years_of_experience,performance_score
0,1,2021-11-25,성북구,보문동5가,37.5838,127.019,강남구,압구정동,37.5297,127.033,...,126.898,0,0,1,29,100,0,1,0,3.96782
1,2,2023-09-22,성북구,안암동4가,37.5807,127.026,강남구,논현동,37.5126,127.028,...,127.080,0,0,1,44,20,0,1,0,4.33979
2,3,2020-05-09,서초구,방배4동,37.4875,126.991,강남구,도곡1동,37.4865,127.040,...,126.864,0,1,1,38,30,0,2,2,4.05321
3,4,2021-04-24,관악구,신사동,37.4840,126.918,강남구,도곡동,37.4901,127.039,...,126.963,1,1,0,17,20,0,1,4,4.44961
4,5,2023-03-11,관악구,봉천동,37.4763,126.953,강남구,논현2동,37.5184,127.037,...,127.020,1,1,1,14,100,1,1,3,4.87076
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1260412,1260413,2020-11-07,서초구,반포본동,37.4994,126.987,강북구,삼각산동,37.6140,127.023,...,127.056,1,0,1,10,100,2,0,0,2.61450
1260413,1260414,2020-03-18,동작구,동작동,37.5005,126.973,강북구,번동,37.6369,127.027,...,127.020,1,1,1,14,100,1,1,3,4.84702
1260414,1260415,2019-10-15,도봉구,도봉동,37.6784,127.043,강북구,수유2동,37.6456,127.018,...,126.915,1,1,1,45,20,1,1,0,3.12101
1260415,1260416,2021-05-29,성북구,안암동5가,37.5881,127.028,강북구,번1동,37.6397,127.030,...,127.026,1,1,0,10,100,6,2,3,4.81536


In [39]:
import pickle
import numpy as np
import pandas as pd
from geopy.distance import geodesic

class DriverMatching:
    def __init__(self, df):
        self.df = df  # 기사 데이터

    # 🚀 **하버사인 거리 계산 함수**
    @staticmethod
    def haversine_distance(coord1, coord2):
        return geodesic(coord1, coord2).kilometers  # km 단위 거리 반환

    # ✅ **기사 추천 함수**
    def recommend_driver(self, user_input):
        # ✅ **1차 필터링: 사용자 조건 충족 기사만 남기기**
        filtered_drivers = self.df[
            ((user_input["cold_storage"] == 0) | (self.df["refrigeration_available"] == 1)) &
            ((user_input["hazardous_material"] == 0) | (self.df["hazardous_material_available"] == 1)) &
            ((user_input["fragile_item"] == 0) | (self.df["fragile_item_available"] == 1)) &
            (self.df["driver_max_delivery_weight"] >= user_input["item_weight"] * user_input["quantity"])
        ].copy()

        # 🚨 예외 처리: 조건에 맞는 기사가 없는 경우
        if filtered_drivers.empty:
            print("🚨 사용자의 조건에 맞는 기사를 찾을 수 없습니다.")
            return None

        # ✅ **출발지 기준 기사와 거리 계산**
        filtered_drivers["distance_to_origin"] = filtered_drivers.apply(
            lambda row: self.haversine_distance(
                (row["driver_latitude"], row["driver_longitude"]),
                (user_input["origin_latitude"], user_input["origin_longitude"])
            ), axis=1
        )

        # ✅ **출발지에서 10km 이내 기사만 남기기**
        filtered_drivers = filtered_drivers[filtered_drivers["distance_to_origin"] <= 10]

        # 기사 수가 부족할 경우 거리 확장
        if filtered_drivers.shape[0] < 10:
            filtered_drivers = filtered_drivers[filtered_drivers["distance_to_origin"] <= 15]
        if filtered_drivers.shape[0] < 5:
            filtered_drivers = filtered_drivers[filtered_drivers["distance_to_origin"] <= 20]

        # ✅ **배달 기록이 있는 기사에게 가산점 부여**
        filtered_drivers["record_match_score"] = np.random.choice([0, 1], size=len(filtered_drivers), p=[0.7, 0.3])

        # ✅ **최종 점수 계산**
        filtered_drivers["final_score"] = (
            filtered_drivers["record_match_score"] * 0.25 +  # 배달 경험 가산점 (25%)
            filtered_drivers["years_of_experience"] * 0.3 +  # 경력 (30%)
            filtered_drivers["kindness"] * 0.2 +  # 친절도 (20%)
            filtered_drivers["score"] * 0.15 +  # 별점 (15%)
            filtered_drivers["communication_encoded"] * 0.1  # 의사소통 점수 (10%)
        )

        # ✅ **최적의 기사 추천 (최고 점수 기사 1명 추천)**
        best_driver = filtered_drivers.sort_values(by="final_score", ascending=False).iloc[0]

        return best_driver

# ✅ **기사 매칭 알고리즘을 Pickle 파일로 저장**
with open("driver_matching_model.pkl", "wb") as file:
    pickle.dump(DriverMatching, file)

print("✅ 기사 매칭 알고리즘(driver_matching_model.pkl) 저장 완료")


✅ 기사 매칭 알고리즘(driver_matching_model.pkl) 저장 완료


In [43]:
# ✅ **저장된 기사 매칭 알고리즘 불러오기**
with open("driver_matching_model.pkl", "rb") as file:
    DriverMatchingClass = pickle.load(file)

# ✅ **기사 매칭 객체 생성**
matching_model = DriverMatchingClass(df)


In [45]:
user_input = {
    "origin_latitude": 37.5, "origin_longitude": 126.9,  # 사용자의 출발지
    "item_weight": 10, "quantity": 1,  # 무게 및 수량
    "cold_storage": 1, "hazardous_material": 0, "fragile_item": 1  # 물품 특성
}

In [47]:
# ✅ **기사 추천 실행**
best_driver = matching_model.recommend_driver(user_input)

# ✅ **추천된 기사 출력**
if best_driver is not None:
    print(" **추천된 기사 정보:**")
    print(f"driverID: {best_driver['driverID']}")
    print(f"distance_to_origin: {best_driver['distance_to_origin']:.2f} km")
    print(f"years_of_experience: {best_driver['years_of_experience']}")
    print(f"kindness: {best_driver['kindness']}")
    print(f"score: {best_driver['score']}")
    print(f"communication_encoded: {best_driver['communication_encoded']}")
    print(f"final_score: {best_driver['final_score']:.2f}")

🚀 **추천된 기사 정보:**
🔹 driverID: 17
🔹 distance_to_origin: 6.97 km
🔹 years_of_experience: 5
🔹 kindness: 5
🔹 score: 5
🔹 communication_encoded: 1
🔹 final_score: 3.60
